# Segmentation Collision detection

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
import os

output_width = 800
output_height = 600
ss_folder = 'D:/Image segmentation/Automatic driving test/ss_collision'

In [2]:
def process_video(video_path, model_path):
    # Load the YOLO model
    model = YOLO(model_path)
    
    # Open the video file or stream
    cap = cv2.VideoCapture(video_path)
    
    # Create the 'ss' folder if it doesn't exist
    if not os.path.exists(ss_folder):
        os.makedirs(ss_folder)
    
    frame_count = 0
    
    while True:
        # Read frame-by-frame
        ret, frame = cap.read()
        
        if not ret:
            break
        
        frame_count += 1
        
        # Perform inference on the frame
        results = model(frame)
        
        # Debug: Print result type and length
        print("Results Type:", type(results))
        print("Number of Results:", len(results))

        # Initialize binary masks
        wheels_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        lanes_mask = np.zeros(frame.shape[:2], dtype=np.uint8)
        
        # Initialize color masks
        wheels_color_mask = np.zeros_like(frame, dtype=np.uint8)
        lanes_color_mask = np.zeros_like(frame, dtype=np.uint8)
        
        if results:
            for result in results:
                if hasattr(result, 'masks') and result.masks:
                    masks = result.masks.data  # Access masks data
                    for mask_idx, mask in enumerate(masks):
                        mask = mask.cpu().numpy().astype(np.uint8)  # Ensure the mask is in the correct format
                        mask = np.squeeze(mask)  # Remove any extra dimensions
                        
                        # Resize mask to match the frame size
                        mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
                        
                        # Check class ID to determine if it's a wheel or lane
                        class_id = result.boxes.cls[mask_idx].item()  # Assuming cls gives the class index
                        if class_id == 1:  # Class ID for 'wheel'
                            wheels_mask = cv2.bitwise_or(wheels_mask, mask_resized)
                            wheels_color_mask[mask_resized > 0] = (0, 255, 0)  # Green for wheels
                        elif class_id == 0:  # Class ID for 'lane'
                            lanes_mask = cv2.bitwise_or(lanes_mask, mask_resized)
                            lanes_color_mask[mask_resized > 0] = (0, 0, 255) 
        
        # Detect collisions by checking overlap between masks
        collision_mask = cv2.bitwise_and(wheels_mask, lanes_mask)
        collision_area = np.sum(collision_mask > 0)

        if collision_area > 100:  # You can adjust the threshold based on your application
            cv2.putText(frame, 'Collision Detected', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            # Save frame with collision
            collision_frame_filename = os.path.join(ss_folder, f'collision_{frame_count:04d}.jpg')
            cv2.imwrite(collision_frame_filename, frame)
        
        # Overlay masks on the original frame
        overlay_frame = cv2.addWeighted(frame, 1.0, wheels_color_mask, 0.5, 0)
        overlay_frame = cv2.addWeighted(overlay_frame, 1.0, lanes_color_mask, 0.5, 0)
        
        # Resize and display the frame
        resized_frame = cv2.resize(overlay_frame, (output_width, output_height))
        cv2.imshow('YOLO Video Detection', resized_frame)
        
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

process_video('D:/Image segmentation/Automatic driving test/videos/car.mp4', 'D:/Image segmentation/Automatic driving test/models/best.pt')


0: 384x640 3 lanes, 1 wheel, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results Type: <class 'list'>
Number of Results: 1

0: 384x640 2 lanes, 1 wheel, 0.0ms
Speed: 0.6ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results Type: <class 'list'>
Number of Results: 1

0: 384x640 2 lanes, 1 wheel, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results Type: <class 'list'>
Number of Results: 1

0: 384x640 2 lanes, 1 wheel, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results Type: <class 'list'>
Number of Results: 1

0: 384x640 2 lanes, 1 wheel, 0.0ms
Speed: 0.0ms preprocess, 0.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
Results Type: <class 'list'>
Number of Results: 1

0: 384x640 2 lanes, 1 wheel, 0.3ms
Speed: 1.8ms preprocess, 0.3ms inference, 0.0ms post